# Deploy a Grafana Dashboard
To track the different stocks on a live dashboard we will use **Grafana**.<br>
We will use [Grafwiz](https://github.com/v3io/grafwiz) to define and deploy the dashboard directly from this notebook

In [1]:
# install prerequisites
%pip install git+https://github.com/v3io/grafwiz --upgrade
%pip install v3io_frames

In [1]:
import mlrun
project = mlrun.get_or_create_project(name='stocks',user_project=True, context="./")

> 2022-11-07 08:12:37,942 [info] loaded project stocks from MLRun DB


In [3]:
import grafwiz
import os
import v3io_frames as v3f

framesd = os.getenv("V3IO_FRAMESD",'framesd:8081')
client = v3f.Client(framesd, container=os.getenv('V3IO_CONTAINER', 'projects'))

grafwiz.DataSource(name='iguazio').deploy('http://grafana',use_auth=True)

dash = grafwiz.Dashboard("stocks", start='now-7d', dataSource='iguazio')

df_columns = ['tickers', 'datetime', 'prediction',  'true']
news_kv = '/stocks-'+ os.environ['V3IO_USERNAME'] + '/artifacts/stocks_prediction'
client.execute("kv",table = news_kv, command = "infer")

tbl = grafwiz.Table(title='Current news Value', span=12,columns=df_columns).source(table=news_kv,fields=df_columns,container='projects')
dash.row([tbl])

dash.deploy('http://grafana')

Datasource iguazio already exists
Datasource iguazio created successfully
Dashboard stocks created successfully
